In [5]:


class Stock:
    
    Exchange='./Data/Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        data = data.astype('float32')
        data = normalize_scale(data) 
        return data

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 

        data=[]#정규화 시작

        for i in range(7):
            crw=[]
            if (i == 4): continue
            for j in range(7):
                
                if(sit[17+i][j+1] != sit[17+i][j+1]):
                    idx =  (float(sit[17+i][j]) + float(sit[17+i][j+2]))/2.0
                    crw.append(idx)
                    continue
                idx = float(sit[17+i][j+1])
                crw.append(idx)
            data.append(normalize(crw))
        data=np.array(data)
        return data
    
    def priceList(self):
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        arr = np.array(cut_csv)
        array = arr[:,1]
        array = array/10000
        array = np.array(array)
        arr = array.reshape(-1,1)
        temp = [[0] * len(arr[0]) for _ in range(len(arr))]
        for i in range(len(arr)):
            temp[i] = arr[len(arr)-1-i]
        temp = np.array(temp).astype(float)
        return temp
    
    def amount(self): #거래량 정규화 된 함수
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        arr = np.array(cut_csv)
        array = arr[:,6]
        n=int(np.mean(array))
        count=1
        while(n != 0):
            n = int(n/10);
            if n == 0 : break
            count=count*10;
        array = arr[:,6]/count
        array = np.array(array)
        arr = array.reshape(-1,1)
        temp = [[0] * len(arr[0]) for _ in range(len(arr))]
        for i in range(len(arr)):
            temp[i] = arr[len(arr)-1-i]
        temp = np.array(temp).astype(float)
        return temp
    def priceamount(self):
        return np.concatenate((self.priceList(), self.amount()), axis=1)
    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]